In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import findspark
import pyspark
import pandas as pd
from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

# https://github.com/kevinschaich/pyspark-cheatsheet
# https://github.com/edyoda/pyspark-tutorial/blob/master/PySpark-DataFrames.ipynb
# https://towardsdatascience.com/7-must-know-pyspark-functions-d514ca9376b9
# https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html    lista med alla metoder
# https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/
# https://www.geeksforgeeks.org/multiple-criteria-for-aggregation-on-pyspark-dataframe/
# https://sparkbyexamples.com/pyspark/pyspark-column-functions/  #visar funktionerna 
# https://stackoverflow.com/questions/52264844/how-get-the-percentage-of-totals-for-each-count-after-a-groupby-in-pyspark  visar withColumn och procent


findspark.init()
findspark.find()


conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


In [3]:
# the csv plays have tabs as deliminator  "\t"
df_play = spark.read.options(header='True', inferSchema='True', delimiter=  "\t").csv("plays.csv")
df_play.show(5)

+--------------------+-------------------+-----------+
|               catch|                 id|     player|
+--------------------+-------------------+-----------+
|2018-02-23 15:22:...|0000-0000-0353-3D35|  darter60k|
|2018-02-23 15:22:...|0000-0000-0353-3D35|davidchofis|
|2018-02-23 15:22:...|0000-0000-0353-3D35| fabioviana|
|2018-02-23 15:22:...|0000-0000-0353-3D35|  gorigokky|
|2018-02-23 15:22:...|0000-0000-0353-3D35|groebenzell|
+--------------------+-------------------+-----------+
only showing top 5 rows



## Count the players that have played the most nbr of times, using 2 ways

In [4]:
# Count the players
df_play.groupBy('player').agg(F.count('*').alias('Frequency')).orderBy('Frequency', ascending=False).show(15)

+----------------+---------+
|          player|Frequency|
+----------------+---------+
| Conducteur59220|     2681|
|        dellbox2|     1951|
|         AE4WiiU|     1686|
|        revolv23|     1437|
|demonhunter47223|     1246|
|    WIIMoustique|     1244|
|buddy1943indiana|     1242|
| Plusbellelavie5|     1234|
|   Feuxdelamoule|     1096|
|      cowboye123|     1068|
|   quentintheret|     1035|
|        Tylano64|     1010|
|       jacky1303|      989|
|         apoc_ze|      958|
|RURI333HRIRU777H|      944|
+----------------+---------+
only showing top 15 rows



In [5]:
# Another way to use agg, but now I can't use alias to rename the column in a good way 
df_play.groupBy('player').agg({'player':'count'}).orderBy('count(player)', ascending = False).show()

+----------------+-------------+
|          player|count(player)|
+----------------+-------------+
| Conducteur59220|         2681|
|        dellbox2|         1951|
|         AE4WiiU|         1686|
|        revolv23|         1437|
|demonhunter47223|         1246|
|    WIIMoustique|         1244|
|buddy1943indiana|         1242|
| Plusbellelavie5|         1234|
|   Feuxdelamoule|         1096|
|      cowboye123|         1068|
|   quentintheret|         1035|
|        Tylano64|         1010|
|       jacky1303|          989|
|         apoc_ze|          958|
|RURI333HRIRU777H|          944|
|      Alexis2012|          919|
|       shinbu-10|          894|
|      shailo1996|          870|
|        Leeshino|          831|
|        666666io|          822|
+----------------+-------------+
only showing top 20 rows



## Create a new date column and view most popular days

### See the most popular days and hours

In [6]:
# Create new columns: Extract the 10 first characters from catch and name it date
# use withColumn method
df_play = df_play.withColumn('date', df_play.catch.substr(0,10))  #make to it is only the date is in one column
df_play = df_play.withColumn('year_month', df_play.catch.substr(0,7))  #make to it is only the year and month are one column
df_play.show()

+--------------------+-------------------+--------------+----------+----------+
|               catch|                 id|        player|      date|year_month|
+--------------------+-------------------+--------------+----------+----------+
|2018-02-23 15:22:...|0000-0000-0353-3D35|     darter60k|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|   davidchofis|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|    fabioviana|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|     gorigokky|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|   groebenzell|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|        heikez|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|   igatake0229|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|     martiwish|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|      risa.lll|2018-02-23|   2018-02|
|2018-02-23 15:22:...|0000-0000-0353-3D3

In [7]:
# Nbr of plays for each day
df_date = df_play.groupBy('date').agg({'date': 'count'}).orderBy('date')
df_date.show()

+----------+-----------+
|      date|count(date)|
+----------+-----------+
|2017-11-16|      82532|
|2017-11-17|       1862|
|2017-11-18|       1447|
|2017-11-19|       1404|
|2017-11-20|       1881|
|2017-11-21|        516|
|2017-11-22|        342|
|2017-11-23|        437|
|2017-11-24|         48|
|2017-12-15|       5948|
|2017-12-16|        472|
|2017-12-17|        370|
|2017-12-18|     976096|
|2017-12-19|       8895|
|2017-12-20|       3016|
|2017-12-21|        416|
|2018-01-02|      43815|
|2018-01-03|       3413|
|2018-01-04|       3477|
|2018-01-05|       3402|
+----------+-----------+
only showing top 20 rows



In [8]:
# The most common hour when a user is playing
# substr(startPosition, lenght)
df_play = df_play.withColumn('hour', df_play.catch.substr(12, 2))
df_play = df_play.withColumn('month', df_play.catch.substr(6, 2))

df_hour = df_play.groupBy('hour').agg({'hour': 'count'}).orderBy('hour')
df_hour.show()

+----+-----------+
|hour|count(hour)|
+----+-----------+
|  00|       6490|
|  01|       5535|
|  02|       5270|
|  03|       4945|
|  04|       5173|
|  05|       5359|
|  06|       6267|
|  07|      29637|
|  08|      25488|
|  09|     366047|
|  10|     898340|
|  11|      29931|
|  12|       7800|
|  13|       8961|
|  14|     747774|
|  15|    1386810|
|  16|     331074|
|  17|      20004|
|  18|      11296|
|  19|       7578|
+----+-----------+
only showing top 20 rows



In [9]:
# Now a lot more of columns regarding date and time have been added
df_play.show()

+--------------------+-------------------+--------------+----------+----------+----+-----+
|               catch|                 id|        player|      date|year_month|hour|month|
+--------------------+-------------------+--------------+----------+----------+----+-----+
|2018-02-23 15:22:...|0000-0000-0353-3D35|     darter60k|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|   davidchofis|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|    fabioviana|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|     gorigokky|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|   groebenzell|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|        heikez|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|   igatake0229|2018-02-23|   2018-02|  15|   02|
|2018-02-23 15:22:...|0000-0000-0353-3D35|     martiwish|2018-02-23|   2018-02|  15|   02|

## How many times a player played for each day

In [10]:
df_play.groupBy('player', 'date').agg(F.count('*').alias('Frequency')).orderBy('Frequency', ascending=False).show(15)

+----------------+----------+---------+
|          player|      date|Frequency|
+----------------+----------+---------+
| Conducteur59220|2018-02-23|     1864|
|    WIIMoustique|2018-02-23|     1069|
| Plusbellelavie5|2018-02-23|      918|
|      cowboye123|2018-02-23|      835|
|   Feuxdelamoule|2018-02-23|      791|
|   quentintheret|2018-02-23|      790|
|        Tylano64|2018-02-23|      757|
|        dellbox2|2018-02-23|      738|
|        dellbox2|2017-12-18|      677|
|      indien1966|2018-02-23|      605|
|          sioux4|2018-02-23|      593|
|        Leeshino|2018-02-23|      584|
|        revolv23|2018-02-23|      583|
|         AE4WiiU|2018-02-23|      576|
|buddy1943indiana|2018-02-23|      556|
+----------------+----------+---------+
only showing top 15 rows



## First day a player playes a level for a month, with 2 methods

Lets see when a player first playes a level for the given month 

First method uses window.partitionBy where I rank each observation and select the first one

The second method is more simple and just uses the method .distinct() that keepes the first observations

In [40]:
# Get number of unique players for each month and the first day they played
# https://stackoverflow.com/questions/60779612/pyspark-get-the-last-observation-in-each-subgroup
# w = Window.partitionBy('year_month', 'player').orderBy(F.desc('year_month'))  # use F.desc to desc orde
w = Window.partitionBy('year_month', 'player').orderBy('catch')


#adding rownumber to the data

# this dataframe is the first for each month
df_first_play = df_play.withColumn("row_nbr",F.row_number().over(w))
df_first_play = df_first_play.where(F.col('row_nbr') == 1)
df_first_play.where(df_play.player == 'Conducteur59220').orderBy('catch', ascending=True).show()

+--------------------+-------------------+---------------+----------+----------+----+-----+-------+
|               catch|                 id|         player|      date|year_month|hour|month|row_nbr|
+--------------------+-------------------+---------------+----------+----------+----+-----+-------+
|2017-11-16 14:45:...|085F-0000-0364-6DA1|Conducteur59220|2017-11-16|   2017-11|  14|   11|      1|
|2017-12-18 14:48:...|000E-0000-0331-2D8D|Conducteur59220|2017-12-18|   2017-12|  14|   12|      1|
|2018-01-03 11:35:...|4935-0000-0376-984D|Conducteur59220|2018-01-03|   2018-01|  11|   01|      1|
|2018-02-01 05:44:...|26EE-0000-0301-E2A0|Conducteur59220|2018-02-01|   2018-02|  05|   02|      1|
|2018-03-01 08:56:...|0019-0000-0369-496E|Conducteur59220|2018-03-01|   2018-03|  08|   03|      1|
|2018-04-10 11:02:...|2860-0000-0396-CD33|Conducteur59220|2018-04-10|   2018-04|  11|   04|      1|
+--------------------+-------------------+---------------+----------+----------+----+-----+-------+


In [41]:
# Compare the method distinct method
# One problem with distinct method is that it looks at all columns, so to get the first month I only can select player and month
df_play.select('player', 'year_month').orderBy('year_month', ascending=False).distinct().where(df_play.player == 'Conducteur59220').show()

+---------------+----------+
|         player|year_month|
+---------------+----------+
|Conducteur59220|   2017-11|
|Conducteur59220|   2018-01|
|Conducteur59220|   2017-12|
|Conducteur59220|   2018-03|
|Conducteur59220|   2018-02|
|Conducteur59220|   2018-04|
+---------------+----------+



In [45]:
# Another way is to use dropDuplicates
df_play.orderBy('catch', ascending=True).dropDuplicates((['year_month', 'player'])).where(df_play.player == 'Conducteur59220').show()

+--------------------+-------------------+---------------+----------+----------+----+-----+
|               catch|                 id|         player|      date|year_month|hour|month|
+--------------------+-------------------+---------------+----------+----------+----+-----+
|2017-12-18 14:48:...|000E-0000-0331-2D8D|Conducteur59220|2017-12-18|   2017-12|  14|   12|
|2018-02-01 05:44:...|26EE-0000-0301-E2A0|Conducteur59220|2018-02-01|   2018-02|  05|   02|
|2018-03-01 08:56:...|0019-0000-0369-496E|Conducteur59220|2018-03-01|   2018-03|  08|   03|
|2018-04-10 11:02:...|2860-0000-0396-CD33|Conducteur59220|2018-04-10|   2018-04|  11|   04|
|2017-11-16 14:45:...|085F-0000-0364-6DA1|Conducteur59220|2017-11-16|   2017-11|  14|   11|
|2018-01-03 11:35:...|4935-0000-0376-984D|Conducteur59220|2018-01-03|   2018-01|  11|   01|
+--------------------+-------------------+---------------+----------+----------+----+-----+



In [46]:
# Get the number of distinct players for each month

w = Window.partitionBy('player').orderBy('year_month')


# this dataframe is the first month only
df_first_month_play = df_play.select('year_month', 'player').withColumn("row_nbr",F.row_number().over(w)).withColumnRenamed('year_month', 'first_month').withColumnRenamed('player', 'player_month')
df_first_month_play = df_first_month_play.where(F.col('row_nbr') == 1)
df_first_month_play.show()

+-----------+---------------+-------+
|first_month|   player_month|row_nbr|
+-----------+---------------+-------+
|    2017-12|      0-0-0E.MN|      1|
|    2017-12|         0-0122|      1|
|    2017-12|         0-0550|      1|
|    2018-02|        0-0jero|      1|
|    2017-12|    0-2-0-6kira|      1|
|    2018-02|        0-2-2-8|      1|
|    2017-12|      0-5-2-8-A|      1|
|    2018-02|         0-6861|      1|
|    2018-03|0-9-0-8NABEKINN|      1|
|    2017-12|      0-BAEZA-0|      1|
|    2018-02|         0-COOL|      1|
|    2017-12|       0-Cypher|      1|
|    2017-12|      0-ETHAN-0|      1|
|    2017-12|       0-HazQ-0|      1|
|    2018-03|0-Hydra_Nexus-0|      1|
|    2018-03| 0-King-Drake-0|      1|
|    2018-03|        0-Robot|      1|
|    2018-03|         0-TK-0|      1|
|    2018-03|   0-Talisker-0|      1|
|    2017-12|    0-Trakeur-0|      1|
+-----------+---------------+-------+
only showing top 20 rows



## Get the cumulative sum for each day, month and hour

There are different ways to calculate the cumulative sum but they all use the function Window to partition over

https://www.datasciencemadesimple.com/cumulative-sum-of-column-and-group-in-pyspark/

https://stackoverflow.com/questions/54134047/pyspark-get-cumulative-sum-of-of-a-column-with-condition


In [54]:
# One way is to create everything in the same code chunk

# notice that the partitionBy is empty, this is because there is no group I want to focus on
cum_sum = df_hour.withColumn('cum_sum', F.sum('count(hour)').over(Window.partitionBy().orderBy().rowsBetween(Window.unboundedPreceding, 0)))
cum_sum.show(25)


+----+-----------+-------+
|hour|count(hour)|cum_sum|
+----+-----------+-------+
|  00|       6490|   6490|
|  01|       5535|  12025|
|  02|       5270|  17295|
|  03|       4945|  22240|
|  04|       5173|  27413|
|  05|       5359|  32772|
|  06|       6267|  39039|
|  07|      29637|  68676|
|  08|      25488|  94164|
|  09|     366047| 460211|
|  10|     898340|1358551|
|  11|      29931|1388482|
|  12|       7800|1396282|
|  13|       8961|1405243|
|  14|     747774|2153017|
|  15|    1386810|3539827|
|  16|     331074|3870901|
|  17|      20004|3890905|
|  18|      11296|3902201|
|  19|       7578|3909779|
|  20|       8316|3918095|
|  21|       8035|3926130|
|  22|       8022|3934152|
|  23|       7226|3941378|
+----+-----------+-------+



In [55]:
# Another way is to first create a variable for the window function making the query code more easily to read
windowval = (Window.partitionBy().orderBy('date').rowsBetween(Window.unboundedPreceding, 0))

df_w_cumsum = df_date.withColumn('cum_sum', F.sum('count(date)').over(windowval))

df_w_cumsum.show()


+----------+-----------+-------+
|      date|count(date)|cum_sum|
+----------+-----------+-------+
|2017-11-16|      82532|  82532|
|2017-11-17|       1862|  84394|
|2017-11-18|       1447|  85841|
|2017-11-19|       1404|  87245|
|2017-11-20|       1881|  89126|
|2017-11-21|        516|  89642|
|2017-11-22|        342|  89984|
|2017-11-23|        437|  90421|
|2017-11-24|         48|  90469|
|2017-12-15|       5948|  96417|
|2017-12-16|        472|  96889|
|2017-12-17|        370|  97259|
|2017-12-18|     976096|1073355|
|2017-12-19|       8895|1082250|
|2017-12-20|       3016|1085266|
|2017-12-21|        416|1085682|
|2018-01-02|      43815|1129497|
|2018-01-03|       3413|1132910|
|2018-01-04|       3477|1136387|
|2018-01-05|       3402|1139789|
+----------+-----------+-------+
only showing top 20 rows



In [68]:
# this show us how many unique players for the month that logged in for the first time that month, lets add a cumulative sum for each month
df_first_play_sum = df_first_play.groupBy('date', 'year_month').sum('row_nbr').orderBy('date')

# The cumulative sum for each day and month
# Notice that I now have a column in partitionBy, this is because there is more than 1 other column
windowval = (Window.partitionBy('year_month').orderBy('date').rowsBetween(Window.unboundedPreceding, 0))

df_first_play_sum.withColumn('cum_sum', F.sum('sum(row_nbr)').over(windowval)).where(F.col('year_month') == '2018-03').show()

+----------+----------+------------+-------+
|      date|year_month|sum(row_nbr)|cum_sum|
+----------+----------+------------+-------+
|2018-03-01|   2018-03|       10810|  10810|
|2018-03-02|   2018-03|      471859| 482669|
|2018-03-03|   2018-03|        2606| 485275|
|2018-03-04|   2018-03|        2222| 487497|
|2018-03-05|   2018-03|        1276| 488773|
|2018-03-06|   2018-03|         965| 489738|
|2018-03-07|   2018-03|        1030| 490768|
|2018-03-08|   2018-03|         891| 491659|
|2018-03-09|   2018-03|        1239| 492898|
|2018-03-10|   2018-03|        1788| 494686|
|2018-03-11|   2018-03|        1558| 496244|
|2018-03-12|   2018-03|         726| 496970|
|2018-03-13|   2018-03|         726| 497696|
|2018-03-14|   2018-03|          32| 497728|
+----------+----------+------------+-------+



## Retention of players

Lets see how many players that played for one month and the drop out

In [82]:
# Create a list with the start day months for players
# This will not be used but is good to know
start_day_list = df_first_play.select('year_month').rdd.map(lambda row : row[0]).distinct().collect()
start_day_list

['2017-11', '2017-12', '2018-01', '2018-03', '2018-02', '2018-04']

In [96]:
# This code was used before, it is used to select the first month a player played
# this will be used as the players first month when calculating the retention rate
w = Window.partitionBy('player').orderBy('year_month')


# this dataframe is the first month only
df_first_month_play = df_play.select('year_month', 'player').withColumn("row_nbr",F.row_number().over(w)).withColumnRenamed('year_month', 'first_month').withColumnRenamed('player', 'player_month')
df_first_month_play = df_first_month_play.where(F.col('row_nbr') == 1)
df_first_month_play.show()

+-----------+---------------+-------+
|first_month|   player_month|row_nbr|
+-----------+---------------+-------+
|    2017-12|      0-0-0E.MN|      1|
|    2017-12|         0-0122|      1|
|    2017-12|         0-0550|      1|
|    2018-02|        0-0jero|      1|
|    2017-12|    0-2-0-6kira|      1|
|    2018-02|        0-2-2-8|      1|
|    2017-12|      0-5-2-8-A|      1|
|    2018-02|         0-6861|      1|
|    2018-03|0-9-0-8NABEKINN|      1|
|    2017-12|      0-BAEZA-0|      1|
|    2018-02|         0-COOL|      1|
|    2017-12|       0-Cypher|      1|
|    2017-12|      0-ETHAN-0|      1|
|    2017-12|       0-HazQ-0|      1|
|    2018-03|0-Hydra_Nexus-0|      1|
|    2018-03| 0-King-Drake-0|      1|
|    2018-03|        0-Robot|      1|
|    2018-03|         0-TK-0|      1|
|    2018-03|   0-Talisker-0|      1|
|    2017-12|    0-Trakeur-0|      1|
+-----------+---------------+-------+
only showing top 20 rows



In [97]:
# To calculate retention I will select the first month a player played as start and then see how many of those players are still active every month
df_player_activity = df_play.select('player', 'year_month').distinct()
df_player_activity.show()

+----------------+----------+
|          player|year_month|
+----------------+----------+
|    ProfAmaretto|   2018-03|
|       recre8ion|   2018-02|
|      LouisWu182|   2018-02|
|     spunkybabes|   2017-12|
|         Stoli23|   2018-03|
|   snorlax082008|   2018-03|
|         lolalao|   2018-03|
|       andrelloo|   2018-03|
|       kougi1974|   2018-04|
|     Boneshark71|   2017-12|
|BernardoVDourado|   2017-12|
|         aas2011|   2018-03|
|    kazkaz160602|   2018-03|
|         Alew110|   2018-02|
|     lalomendoza|   2017-12|
|       opperkoek|   2018-02|
|       joelmaria|   2018-03|
|      sateraaito|   2018-02|
|      hiroki1682|   2018-03|
|         MAEKEN1|   2018-02|
+----------------+----------+
only showing top 20 rows



In [98]:
# Combine the df with all the players distinct activites for all months and add a column that shows which month that is the first 

df_retention = df_player_activity.join(df_first_month_play , df_player_activity.player == df_first_month_play.player_month, 'left').select('player', 'year_month', 'first_month')
df_retention.orderBy('player').show()

+---------------+----------+-----------+
|         player|year_month|first_month|
+---------------+----------+-----------+
|      0-0-0E.MN|   2017-12|    2017-12|
|      0-0-0E.MN|   2018-02|    2017-12|
|         0-0122|   2017-12|    2017-12|
|         0-0122|   2018-03|    2017-12|
|         0-0550|   2017-12|    2017-12|
|         0-0916|   2018-02|    2018-02|
|        0-0jero|   2018-02|    2018-02|
|    0-2-0-6kira|   2017-12|    2017-12|
|        0-2-2-8|   2018-02|    2018-02|
|      0-5-2-8-A|   2017-12|    2017-12|
|      0-5-2-8-A|   2018-03|    2017-12|
|      0-5-2-8-A|   2018-02|    2017-12|
|         0-6861|   2018-02|    2018-02|
|0-9-0-8NABEKINN|   2018-03|    2018-03|
|      0-BAEZA-0|   2017-12|    2017-12|
|         0-COOL|   2018-03|    2018-02|
|         0-COOL|   2018-02|    2018-02|
|       0-Cypher|   2017-12|    2017-12|
|      0-ETHAN-0|   2018-03|    2017-12|
|      0-ETHAN-0|   2018-02|    2017-12|
+---------------+----------+-----------+
only showing top

In [95]:
# lets take a look at the famous player Conducteur59220 and control that it looks correct 
# Now there is one column (year_month) that shows every month that Conducteur59220 have been playing
# And one column (first_month) that shows the first month only, now it will be easy to calculate the retention rate
df_retention.where(F.col('player') == 'Conducteur59220').show()

+---------------+----------+-----------+
|         player|year_month|first_month|
+---------------+----------+-----------+
|Conducteur59220|   2017-11|    2017-11|
|Conducteur59220|   2018-01|    2017-11|
|Conducteur59220|   2017-12|    2017-11|
|Conducteur59220|   2018-03|    2017-11|
|Conducteur59220|   2018-02|    2017-11|
|Conducteur59220|   2018-04|    2017-11|
+---------------+----------+-----------+



In [100]:
# Code to create the retention data, all that is left is to pivot the table so nbr of months are the columns and first month is just one row
# The code counts how many times players that occurs for the first month and then create a column with the max value
# I then created a new column with the max value of players, since the first month will always have the max value or be equal 
# The retention rate is then calculated as the nbr of players for a given month / max players (players in the first month cohort)

df_retention = df_retention.groupBy('first_month', 'year_month' ).agg(F.count('player').alias('nbr_players'))\
  .withColumn("max_players",  F.max("nbr_players").over(Window.partitionBy('first_month'))).orderBy('first_month', 'year_month') \
  .withColumn("retantion_rate", F.col("nbr_players") / F.max("nbr_players").over(Window.partitionBy('first_month'))).orderBy('first_month', 'year_month')\
  .withColumn('row_num', F.row_number().over(Window.partitionBy('first_month').orderBy('year_month')))\
  .withColumn("nbr_month", F.concat(F.lit("month_"), F.col("row_num")))

df_retention.show()

+-----------+----------+-----------+-----------+--------------------+-------+---------+
|first_month|year_month|nbr_players|max_players|      retantion_rate|row_num|nbr_month|
+-----------+----------+-----------+-----------+--------------------+-------+---------+
|    2017-11|   2017-11|      66199|      66199|                 1.0|      1|  month_1|
|    2017-11|   2017-12|      47139|      66199|  0.7120802429039713|      2|  month_2|
|    2017-11|   2018-01|       8445|      66199| 0.12756990286862338|      3|  month_3|
|    2017-11|   2018-02|      49131|      66199|   0.742171331893231|      4|  month_4|
|    2017-11|   2018-03|      48842|      66199|  0.7378057070348495|      5|  month_5|
|    2017-11|   2018-04|       1114|      66199| 0.01682804876206589|      6|  month_6|
|    2017-12|   2017-12|     358117|     358117|                 1.0|      1|  month_1|
|    2017-12|   2018-01|      23051|     358117| 0.06436723193816546|      2|  month_2|
|    2017-12|   2018-02|     196

In [101]:
# Use the table above to pivot it to nbr_month as columns
# This table shows the retentation rate given the first month as cohort 
df_retention.groupBy('first_month').pivot('nbr_month').sum('retantion_rate').show()

+-----------+-------+--------------------+--------------------+--------------------+--------------------+-------------------+
|first_month|month_1|             month_2|             month_3|             month_4|             month_5|            month_6|
+-----------+-------+--------------------+--------------------+--------------------+--------------------+-------------------+
|    2017-11|    1.0|  0.7120802429039713| 0.12756990286862338|   0.742171331893231|  0.7378057070348495|0.01682804876206589|
|    2017-12|    1.0| 0.06436723193816546|  0.5499152511609334|  0.5414291977202981|0.004962065470223418|               null|
|    2018-01|    1.0|  0.5445921266233766|  0.5396205357142857|0.013367491883116884|                null|               null|
|    2018-02|    1.0|  0.3589235733738413|0.003713697037854532|                null|                null|               null|
|    2018-03|    1.0|0.003880337150685...|                null|                null|                null|             